# ETL para la tabla review
 Se recorre el archivo "review.json" para construir nuevos archivos .parque  
  clasificando por año, se identifico que la data va desde 2005 hasta 2022, 
  por lo tanto se tendran 17 archivos de la forma ... review2019.csv, review2020.csv ... 
 nota = En la carpeta donde se ejecuta debe estar el archivo "review.json" y 
        no deben estar archivos review20XX.csv de corridas anteriores

In [1]:
#Librerias a utilizar
import pandas as pd 
from IPython.display import clear_output
import os 
from datetime import datetime

In [2]:
#Diseñado para leer el archivo con el nombre indicado para carga incremental 
# simplemente colocar el archivo .json con el nombre indicado.

nombre = 'review' 
path = os.path.abspath(os.getcwd())+"\\review_"+datetime.now().strftime("%d_%m_%Y_%H-%M")

# Check whether the specified path exists or not
if not os.path.exists(path):
    # Create a new directory because it does not exist 
  os.makedirs(path,exist_ok=False)
  print("The new directory is created!")

The new directory is created!


In [3]:
# Organizar la informacion Carga de datos
# Se recorre el archivo "review.json" para construir nuevos .csv  
#  clasificando por año, se identifico que la data va desde 2005 hasta 2022, 
#  por lo tanto se tendran 17 archivos de la forma ... review2019.csv, review2020.csv ... 
# nota = En la carpeta donde se ejecuta debe estar el archivo "review.json" y 
#        no deben estar archivos review20XX.csv de corridas anteriores
#Tiempo de ejecucion aprox 15min si se elimina la limpieza final (eliminar ducplicados) <9min

import pandas as pd 
from IPython.display import clear_output
import os 

def cargaIncremental(df,nombre ='review'):
    df['date'] = pd.to_datetime(df['date'])
    for year in range(2005,2023):
        if os.path.isfile(path+'\\'+nombre+"_"+str(year)+".csv"):
            #Carga incremental de datos en .csv
            print('actualizando '+path+'\\'+nombre+"_"+str(year)+".csv")
            filtered_df = df.loc[df['date'].dt.year == year]
            filtered_df.to_csv(path+'\\'+nombre+"_"+str(year)+".csv",mode='a', index=False, header=False)
        else:
            #Primera corrida
            print('creando '+nombre+"_"+str(year)+".csv")
            filtered_df = df.loc[df['date'].dt.year == year]
            filtered_df.to_csv(path+'\\'+nombre+"_"+str(year)+".csv",index=False)

total = 0 
cont = 0
cont_max = 35  #Se sabe que tamaño +200k  se haran 35 partes (chunk) del .json 

Tamano = 200000
# Se recorre por partes el archivo .json
for chunk in pd.read_json(nombre+".json",chunksize=Tamano, lines=True):
        # Visualizacion de avance
        cont = cont + 1
        clear_output(wait=True)
        print('Completado Etapa 1: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
        
        #Se carga la parte
        cargaIncremental(chunk)

#Tiempo esperado de corrida: 8 min.

Completado Etapa 1: 35/35  100.0%
actualizando c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2005.csv
actualizando c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2006.csv
actualizando c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2007.csv
actualizando c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2008.csv
actualizando c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2009.csv
actualizando c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2010.csv
actualizando c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2011.csv
actualizando c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2012.csv
actualizando c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2013.csv
actualizando c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2014.csv
actualizando c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_202

In [9]:
#Borrando duplicados en caso de que existan
cont = 0
cont_max = 18 
#Limpieza final
for year in range(2005,2023):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado Etapa 2: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    if os.path.isfile(path+'\\'+nombre+"_"+str(year)+".csv"):
        #Revision de datos en .csv
        df = pd.read_csv(path+'\\'+nombre+"_"+str(year)+".csv") 
        antes = df.shape[0]
        filtered_df = df.drop_duplicates().reset_index(drop=True)
        despues = filtered_df.shape[0]
        print(nombre+"_"+str(year)+".csv cambio de",antes," a ",despues,"columnas, se perdieron ",antes-despues,"columnas")
        filtered_df.to_csv(path+'\\'+nombre+"_"+str(year)+".csv",index=False)
        
#Tiempo: 2 min si se hace despues de borrar la columna texto, sino tarda como 30 min. 

Completado Etapa 2: 18/18  100.0%
review_2022.csv cambio de 31665  a  31665 columnas, se perdieron  0 columnas


In [4]:
#Manejo de columna Texto
#tiempo aprox 
def ETL_texto(df):
    #Por ahora solo se borrara la columna texo
    df.drop(columns=["text"],inplace=True)

cont = 0
cont_max = 18 
#Limpieza final
for year in range(2005,2023):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado Etapa: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    if os.path.isfile(path+'\\'+nombre+"_"+str(year)+".csv"):
        #Revision de datos en .csv
        df = pd.read_csv(path+'\\'+nombre+"_"+str(year)+".csv") 
        df.drop(columns=["text"],inplace=True)
        df.to_csv(path+'\\'+nombre+"_"+str(year)+".csv",index=False)
#Tiempo: 3 min

Completado Etapa: 18/18  100.0%


In [12]:
#Manejo de la columna user_id para cual se necesita el archivo codes_users.csv 
# creado desde "user.json" ....
#users = pd.read_json("user.json", lines=True , chunksize=200000) 
#cods_users = pd.concat(users).user_id
#cods_users = cods_users.reset_index()
#cods_users = cods_users.rename(columns={'index':'n_user_id'})
#cods_users.to_csv(path+'\\'+'codes_users.csv', index=False)
#tiempo: 3min 

def  cambio_Iduser(nombre_csv,derecha):
    izquierda = pd.read_csv(nombre_csv) 
    result = izquierda.join(derecha, on='user_id')
    result = result[['review_id', 'n_user_id', 'business_id', 'stars', 'useful', 'funny','cool', 'date']]
    result = result.rename(columns={"n_user_id": "Iduser"})
    print('actualizando Iduser de '+nombre_csv)
    result.to_csv(nombre_csv,index=False)

# Para visualizacion de avance
cont = 0
cont_max = 18 

#Preparando join left
codes_users = pd.read_csv('codes_users.csv')
derecha = codes_users.set_index('user_id')

for year in range(2005,2023):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    #Cambio 'userId'(str) en cada archivo reviewXXXX.csv por Iduser(int)
    if os.path.isfile(path+'\\'+nombre+"_"+str(year)+".csv"):
        cambio_Iduser(path+'\\'+nombre+"_"+str(year)+".csv",derecha)
 
#tiempo: 2 min 

Completado: 18/18  100.0%
actualizando Iduser de c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2022.csv


In [14]:
#Manejo de la columna business_id para cual se necesita el archivo Ibusiness_index.csv que 
# se obtiene de business.json ....
#df_business = pd.read_json('business.json', lines=True)
#business_index = pd.DataFrame()
#business_index['id'] = list(range(0, len(df_business) + 1, 1))
#business_index['business_id'] = df_business['business_id'].copy()
#business_index.to_csv("business_index.csv",index=False)

def  cambio_Idbusiness(nombre_csv,derecha):
    izquierda = pd.read_csv(nombre_csv) 
    result = izquierda.join(derecha, on='business_id')
    result = result[['review_id', 'Iduser', 'id', 'stars', 'useful', 'funny','cool', 'date']]
    result = result.rename(columns={"id": "Idbusiness"})
    print('actualizando Iduser de '+nombre_csv)
    result.to_csv(nombre_csv,index=False)

# Para visualizacion de avance
cont = 0
cont_max = 18 

#Preparando join left
business_index = pd.read_csv("business_index.csv")
derecha = business_index.set_index('business_id')

for year in range(2005,2023):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    #Cambio 'userId'(str) en cada archivo reviewXXXX.csv por Iduser(int)
    if os.path.isfile(path+'\\'+nombre+"_"+str(year)+".csv"):
        cambio_Idbusiness(path+'\\'+nombre+"_"+str(year)+".csv",derecha)

#tiempo: 2 min

Completado: 18/18  100.0%
actualizando Iduser de c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2022.csv


In [16]:
# Manejo de la columna Idreview 
# Codigo para ir leyendo los df "review_20xx.csv" e ir creando la tabla Idreview.csv

def  creacion_Idreview(nombre_csv,indice=0):
    df = pd.read_csv(nombre_csv) 
    
    if os.path.isfile(path+'\\Idreview.csv'):
        #Carga incremental de datos 
        print('actualizando Idreview.csv con '+nombre_csv)
        #Se crea el nuevo "Id_review" en secuencia con anterior
        lista = list(range(indice+1,indice+df.shape[0]+1))
        nuevo_indice = lista[-1]
        
        df.insert(0,"Id_review",lista)
        Idreview = df[["Id_review","review_id"]]
        df.drop(columns=["review_id"],inplace=True)
        df.to_csv(nombre_csv,index=False)
        Idreview.to_csv(path+'\\Idreview.csv',mode='a', index=False, header=False)
    else:
        #Primera corrida
        print('creando Idreview.csv')
        df.reset_index(inplace=True)
        df = df.rename(columns={"index": "Id_review"})
        Idreview = df[["Id_review","review_id"]]
        df.drop(columns=["review_id"],inplace=True)
        df.to_csv(nombre_csv,index=False)
        Idreview.to_csv(path+'\\Idreview.csv',index=False)
        nuevo_indice =  Idreview.iloc[-1][0]               
    return nuevo_indice


cont = 0
cont_max = 18 
indice = 0
for year in range(2005,2023):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    #Creacion de tabla 'Idreview.csv' y cambio de "review_id"(str)  por "Id_review"(int64)
    if os.path.isfile(path+'\\'+nombre+"_"+str(year)+".csv"):
        indice = creacion_Idreview(path+'\\'+nombre+"_"+str(year)+".csv",indice)

#tiempo: 2 min 

Completado: 18/18  100.0%
actualizando Idreview.csv con c:\Henry\LABS_01\221114_GroupLab\JHO\review_23_11_2022_15-05\review_2022.csv


In [19]:
df = pd.read_csv(path+'\\review_2019.csv')
df.head()

,Id_review,Iduser,Idbusiness,stars,useful,funny,cool,date
0,4878585,143867,8313,5,0,0,0,2019-01-04 02:18:09
1,4878586,155207,11427,5,0,0,0,2019-01-06 11:48:21
2,4878587,84622,9889,5,0,0,0,2019-01-27 15:08:14
3,4878588,1210,9419,4,2,0,1,2019-02-17 20:28:26
4,4878589,190019,2376,1,0,0,0,2019-02-17 17:17:56


In [20]:
df = pd.read_csv(path+'\\Idreview.csv')
df.head()

,Id_review,review_id
0,0,ep8FfviAfqHNyhpW6MQ2jA
1,1,9eeoAUQEJDw0KYV5NPYH-A
2,2,RcsyeHfZqpGdb9z-AkUdSA
3,3,JFjQ5RntPGhsNO2agAEYtg
4,4,MSt3hkzLBmIy8b8VwXlC7Q


In [21]:
#Primera corrida:
#Se presentan los resultados finales en formato parquet en carpeta donde se esta corriendo el script
# Para visualizacion de avance 
cont = 0
cont_max = 18 
for year in range(2005,2023):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    if os.path.isfile(path+'\\'+nombre+"_"+str(year)+".csv"):
        df =pd.read_csv(path+'\\'+nombre+"_"+str(year)+".csv")
        df.to_parquet(nombre+"_"+str(year)+'_parquet.gzip',compression='gzip')

#conversion a parquet para que pese menos al almacenar
df =pd.read_csv(path+'\\Idreview.csv')
df.to_parquet('Idreview_parquet.gzip',compression='gzip') 


Completado: 18/18  100.0%


In [ ]:
#En caso de hacer otra corrida con otro archivo .json se debe anexar la data que esta en la 
# carpeta path a la que se tanga anteriormente:
cont = 0
cont_max = 18 
for year in range(2005,2023):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    if os.path.isfile(path+'\\'+nombre+"_"+str(year)+".csv"):
        df =pd.read_csv(path+'\\'+nombre+"_"+str(year)+".csv")
        ## por revisar porque tambien se puede hacer directamente en la carga MySQL 
 
df =pd.read_csv(path+'\\Idreview.csv')
    ## por revisar porque tambien se puede hacer directamente en la carga MySQL 


In [27]:
# Se borran archivos innecesarios en caso de requerirlo:
from os import remove
from os import rmdir
cont = 0
cont_max = 18
for year in range(2005,2023):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    if os.path.isfile(path+'\\'+nombre+"_"+str(year)+".csv"):
        remove(path+'\\'+nombre+"_"+str(year)+".csv")
        
if os.path.isfile(path+'\\Idreview.csv'):
    remove(path+'\\Idreview.csv')
rmdir(path)

Completado: 18/18  100.0%


In [ ]:
#al final de la corrida deben quedar los archivos reviewXXXX_parquet.gzip para armar la base de datos
# y el archivo Idreview_parquet.gzip